In [1]:
from ultralytics import YOLO
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
datasets_path = os.getenv('DATASET_PATH_YOLO')
datasets_annotations_path = os.getenv('DATASET_ANNOTATIONS_PATH_YOLO')
datasets_images_path = os.getenv('DATASET_IMAGES_PATH_YOLO')

## Dataset Checkings

In [3]:
import json

# Load JSON file
json_path_train = datasets_annotations_path + "/train.json"  # Ubah ke lokasi file JSON Anda
with open(json_path_train, "r") as f:
    data = json.load(f)

# Print some information
print("Jumlah gambar:", len(data["images"]))
print("Jumlah anotasi:", len(data["annotations"]))
print("Kategori:", data["categories"])

# Check a sample image and annotation
print("\nContoh data gambar:")
print(data["images"][0])

print("\nContoh data anotasi:")
print(data["annotations"][0])

Jumlah gambar: 596
Jumlah anotasi: 1003
Kategori: [{'id': 1, 'name': 'damage', 'supercategory': 'none'}]

Contoh data gambar:
{'id': 0, 'license': 1, 'file_name': '155_jpg.rf.06593f538171c239122303ed08a56d3b.jpg', 'height': 168, 'width': 300, 'date_captured': '2022-04-21T04:47:27+00:00'}

Contoh data anotasi:
{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [73, 21, 75, 55], 'area': 4125, 'segmentation': [[77, 75, 105, 76, 137, 63, 148, 47, 136, 27, 115, 21, 97, 25, 84, 35, 73, 49]], 'iscrowd': 0}


In [4]:
import json

# Load JSON file
json_path_val = datasets_annotations_path + "/val.json"  # Ubah ke lokasi file JSON Anda
with open(json_path_val, "r") as f:
    data = json.load(f)

# Print some information
print("Jumlah gambar:", len(data["images"]))
print("Jumlah anotasi:", len(data["annotations"]))
print("Kategori:", data["categories"])

# Check a sample image and annotation
print("\nContoh data gambar:")
print(data["images"][0])

print("\nContoh data anotasi:")
print(data["annotations"][0])

Jumlah gambar: 196
Jumlah anotasi: 341
Kategori: [{'id': 1, 'name': 'damage', 'supercategory': 'none'}]

Contoh data gambar:
{'id': 0, 'license': 1, 'file_name': '160_jpg.rf.697c4d606e9f15b7ac8506e27077e541.jpg', 'height': 392, 'width': 720, 'date_captured': '2022-04-21T04:47:27+00:00'}

Contoh data anotasi:
{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [419, 168, 248, 169], 'area': 41912, 'segmentation': [[509, 337, 573, 335, 641, 335, 658, 308, 665, 279, 667, 253, 667, 215, 663, 169, 609, 168, 567, 169, 542, 183, 475, 224, 425, 293, 421, 317, 419, 329]], 'iscrowd': 0}


In [5]:
import json
import os
from PIL import Image  # Impor pustaka untuk membuka gambar

def convert_coco_to_yolo(coco_json_path, output_dir, image_dir):
    # Buka file JSON COCO
    with open(coco_json_path) as f:
        coco_data = json.load(f)
    
    # Buat folder output jika belum ada
    os.makedirs(output_dir, exist_ok=True)

    # Ambil informasi kategori (class)
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    category_id_map = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}

    # Iterasi melalui anotasi
    annotations = coco_data['annotations']
    images = {img['id']: img['file_name'] for img in coco_data['images']}
    
    for annotation in annotations:
        image_id = annotation['image_id']
        file_name = images[image_id]
        
        # Path ke gambar
        img_path = os.path.join(image_dir, file_name)
        
        # Buka gambar untuk mendapatkan dimensi
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        # File label output
        label_file = os.path.join(output_dir, file_name.replace('.jpg', '.txt'))
        
        # Koordinat bounding box
        bbox = annotation['bbox']
        x_min, y_min, width, height = bbox
        x_center = x_min + width / 2
        y_center = y_min + height / 2
        
        # Normalisasi (dalam skala 0-1)
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        # ID kategori yang diubah ke indeks YOLO
        category_id = annotation['category_id']
        yolo_category_id = category_id_map[category_id]
        
        # Tulis ke file
        with open(label_file, 'a') as lf:
            lf.write(f"{yolo_category_id} {x_center} {y_center} {width} {height}\n")

    print("Konversi selesai.")


In [6]:
convert_coco_to_yolo(
    coco_json_path=json_path_train,
    output_dir=datasets_path + "/labels/train",
    image_dir=datasets_path + "/images/train"
)

Konversi selesai.


In [7]:
convert_coco_to_yolo(
    coco_json_path=json_path_val,
    output_dir=datasets_path + "/labels/val",
    image_dir=datasets_path + "/images/val"
)

Konversi selesai.


In [8]:
import os

for file in os.listdir(datasets_path + "/labels/train"):
    if not file.endswith(".txt"):
        continue
    with open(os.path.join(datasets_path + "/labels/train", file), 'r') as f:
        lines = f.readlines()
        if not lines:
            print(f"Empty file: {file}")
        for line in lines:
            values = line.strip().split()
            if len(values) != 5:
                print(f"Incorrect format in file {file}: {line}")

In [9]:
import os

for file in os.listdir(datasets_path + "/labels/val"):
    if not file.endswith(".txt"):
        continue
    with open(os.path.join(datasets_path + "/labels/val", file), 'r') as f:
        lines = f.readlines()
        if not lines:
            print(f"Empty file: {file}")
        for line in lines:
            values = line.strip().split()
            if len(values) != 5:
                print(f"Incorrect format in file {file}: {line}")

In [10]:
model = YOLO('yolov8n.pt')         

100%|██████████| 6.25M/6.25M [00:18<00:00, 358kB/s]


In [ ]:
model.train(
    data="dataset.yaml",
    epochs=50,
    batch=8,
    imgsz=512,
)

New https://pypi.org/project/ultralytics/8.3.56 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.54  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\labels\train... 596 images, 29 backgrounds, 1 corrupt: 100%|██████████| 625/625 [00:00<00:00, 1566.26it/s]

train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\100_jpg.rf.ec010f806c856b308ad7d050880c39fa.jpg: 8 duplicate labels removed
train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\101_jpg.rf.05a9c7e86de33e0e9c08395bd522b0f3.jpg: 4 duplicate labels removed
train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\102_jpg.rf.6cda08a5ad0a691b55b342d26f73a158.jpg: 4 duplicate labels removed
train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\103_jpg.rf.3324757de21a8b5087e84725cff6957f.jpg: 4 duplicate labels removed
train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\104_jpg.rf.aff208134de1a6df75660d1fde5f0b0d.jpg: 8 duplicate labels removed
train: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\train\105_jpg.rf.cc2b4d77aa3e6ae0b6975933b60fec3c.jpg: 8 duplicate labels 


val: Scanning D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\labels\val... 196 images, 12 backgrounds, 0 corrupt: 100%|██████████| 208/208 [00:00<00:00, 882.85it/s]

val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\114_jpg.rf.cfabf01d2e3f751396eaa9660070fd48.jpg: 4 duplicate labels removed
val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\123_jpg.rf.bdecf2e72e09ddfc9ff20d5334144e23.jpg: 8 duplicate labels removed
val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\132_jpg.rf.0117b03b1bb88f7edf293a704383350f.jpg: 4 duplicate labels removed
val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\14_jpg.rf.17d0950f1a5740f28968b97288aa9285.jpg: 4 duplicate labels removed
val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\15_jpg.rf.34119d478a347be72f231a096f078449.jpg: 4 duplicate labels removed
val: WARNING  D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Yolov8\images\val\160_jpg.rf.697c4d606e9f15b7ac8506e27077e541.jpg: 4 duplicate labels removed
val: WARNING  D:\D

...

                   all        208        341      0.423      0.349      0.265     0.0984



50 epochs completed in 0.132 hours.
Optimizer stripped from runs\detect\train26\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train26\weights\best.pt, 6.2MB

Validating runs\detect\train26\weights\best.pt...
Ultralytics 8.3.54  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00, 10.59it/s]


                   all        208        341      0.453      0.317      0.259     0.0993
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train26


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F38CCC63E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [13]:
results = model.predict(source=datasets_images_path + "/val", save=True, conf=0.25)


image 1/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\114_jpg.rf.cfabf01d2e3f751396eaa9660070fd48.jpg: 480x640 1 Car-Damage, 107.3ms
image 2/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\123_jpg.rf.bdecf2e72e09ddfc9ff20d5334144e23.jpg: 480x640 2 Car-Damages, 79.1ms
image 3/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\132_jpg.rf.0117b03b1bb88f7edf293a704383350f.jpg: 480x640 1 Car-Damage, 9.0ms
image 4/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\14_jpg.rf.17d0950f1a5740f28968b97288aa9285.jpg: 480x640 1 Car-Damage, 11.5ms
image 5/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\15_jpg.rf.34119d478a347be72f231a096f078449.jpg: 480x640 2 Car-Damages, 10.0ms
image 6/208 D:\Datasets\riset_infor_dataset\Car_Damage_Coco_Dataset_Combined\images\val\160_jpg.rf.697c4d606e9f15b7ac8506e27077e541.jpg: 352x640 1 Car-Damage, 51.9ms
ima